Creates side by side images of input and target image for the tensorflow pix2pix model.

In [ ]:
#%tensorflow_version 1.4.1

#import tensorflow
#print(tensorflow.__version__)

import os
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
orig_dir = '/content/gdrive/My Drive/NEUTROPHIL/Train/240/240'
print('total images:', len(os.listdir(orig_dir)))
temp = os.listdir(orig_dir)

In [ ]:
mag_dir = '/content/gdrive/My Drive/NEUTROPHIL/Train/240/240'
print('total images:', len(os.listdir(mag_dir)))

In [ ]:
import numpy as np
from numpy import isneginf
from scipy.stats import iqr 

def compute_mag_phase(toBeTransfromed):
    dft = np.fft.fft2(toBeTransfromed)
    dft_shift = np.fft.fftshift(dft)
    mag = np.abs(dft_shift)
    ang = np.angle(dft_shift)
    return mag, ang

def reconstruct(mag,ang):
    combined = np.multiply(mag, np.exp(1j*ang))
    fftx = np.fft.ifftshift(combined)
    ffty = np.fft.ifft2(fftx)
    imgCombined = np.abs(ffty)
    return imgCombined

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (240 + 240, 240))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

save_dir = '/content/gdrive/My Drive/NEUTROPHIL/Train/pix2pix_input3'
next_pix = [os.path.join(orig_dir, fname) 
                for fname in temp[:]]

mag_pix = os.path.join(mag_dir, 'A_100_1346.jpeg')
img_mag = Image.open(mag_pix)
im_mag = np.array(img_mag) /255
mag_mag,ang_mag = compute_mag_phase(im_mag)

for i, img_path in enumerate(next_pix):
  img = Image.open(img_path)
  im = np.array(img) /255
  mag,ang = compute_mag_phase(im)
  recon = reconstruct(mag,ang_mag)
  recon = np.clip(recon,0,1)
  #if i ==5:
  #  plt.imshow(recon)
  im = Image.fromarray((recon * 255).astype(np.uint8),'RGB')
  #if i ==5:
  #  plt.imshow(im)
  save_dest = os.path.join(save_dir, temp[i])
  #save_dest = save_dest + '.png'
  get_concat_h(im, img).save(save_dest)